In [1]:
import json
from pathlib import Path
import numpy as np
from collections import OrderedDict
from ase.build.molecule import molecule
from hilde.helpers.fileformats import append_to_json_array, to_json, from_json
from hilde.templates.aims import setup_aims
from ase.calculators.emt import EMT
from ase.md.verlet import VelocityVerlet
from ase import units as u
from pprint import pprint

In [2]:
h2o = molecule('H2O')
h2o.positions[0, 0] += .05
h2o.positions

array([[ 0.05    ,  0.      ,  0.119262],
       [ 0.      ,  0.763239, -0.477047],
       [ 0.      , -0.763239, -0.477047]])

In [3]:
def md2dict(obj):
    dct = OrderedDict()
    dct['nsteps'] = obj.nsteps
    dct['dt'] = obj.dt
    dct['masses'] = obj.masses.T.tolist()[0]
    
    return dct

In [4]:
def atoms2dict(atoms):
    atoms_dict = OrderedDict()
    
    if any(atoms.pbc):
        atoms_dict['cell'] = atoms.cell
    atoms_dict['positions'] = atoms.positions
    
    return atoms_dict

In [5]:
def calc2dict(calc):
    calc_dict = OrderedDict()
    
    calc_dict = calc.results
    
    return calc_dict

In [6]:
def step2dict(atoms, md):
    md_dict = md2dict(md)
    
    atoms_dict = atoms2dict(atoms)
    calc_dict = calc2dict(atoms.calc)
    
    return OrderedDict({'MD': md_dict, 'atoms': atoms_dict, 'calculator': calc_dict})

In [7]:
def metadata2dict(atoms, calc, md):
    md_dict = md.todict()
    md_dict['fs'] = u.fs
    
    atoms_dict = atoms2dict(atoms)
    atoms_dict['numbers'] = atoms.numbers
    atoms_dict['symbols'] = [f'{sym}' for sym in atoms.symbols]
    
    calc_dict = OrderedDict()
    calc_dict['name'] = calc.__class__.__name__
    calc_dict['params'] = calc.todict()
    
    meta_dict = OrderedDict({
        'atoms': atoms_dict,
        'md': md_dict,
        'calculator': calc_dict,
    })
    
    return meta_dict

### Initialize MD and calculator

In [8]:
h2o.calc = EMT()
md = VelocityVerlet(h2o, timestep=u.fs)

### Save metadata and initialize trajectory

In [9]:
metadata = metadata2dict(h2o, h2o.calc, md)
to_json(metadata, 'md_metadata.json')

trajectory = Path('trajectory.json')
if trajectory.exists():
    trajectory.unlink()    

### Run MD

In [10]:
for _ in range(2):
    md.run(1)
    append_to_json_array(step2dict(h2o, md), trajectory, indent=1)

In [11]:
traj = from_json(trajectory)

In [12]:
for st in traj:
    print(st['MD']['nsteps'])

1
2
